In [1]:
from math import log10, sqrt
import time
import os
from os.path import join
import import_ipynb
import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from model import VDSR
from dataset import DF2K
import matplotlib.pyplot as plt
import config
from torchvision import transforms

importing Jupyter notebook from model.ipynb
importing Jupyter notebook from dataset.ipynb
importing Jupyter notebook from config.ipynb


In [2]:
def save_loss_plot(out_dir, train_loss, val_loss):
    figure_1, train_ax = plt.subplots()
    figure_2, valid_ax = plt.subplots()
    train_ax.plot(train_loss, color='tab:blue')
    train_ax.set_xlabel('iterations')
    train_ax.set_ylabel('train loss')
    valid_ax.plot(val_loss, color='tab:red')
    valid_ax.set_xlabel('iterations')
    valid_ax.set_ylabel('validation loss')
    figure_1.savefig(f"{out_dir}/train_loss.png")
    figure_2.savefig(f"{out_dir}/valid_loss.png")
    print('SAVING PLOTS COMPLETE...')
    
    plt.close('all')

In [3]:
plt.style.use('ggplot')

In [4]:
input_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomResizedCrop((config.INPUT_IMAGE_HEIGHT, config.INPUT_IMAGE_WIDTH)),
    transforms.RandomHorizontalFlip(),
]
)

target_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((config.INPUT_IMAGE_HEIGHT, config.INPUT_IMAGE_WIDTH)),]
)

In [5]:
trainDS = DF2K(
    imagePaths=config.TRAIN_DATASET_PATH, 
    input_transforms=input_transforms, 
    target_transforms=target_transforms,
)
validDS = DF2K(
    imagePaths=config.VALID_DATASET_PATH, 
    input_transforms=input_transforms, 
    target_transforms=target_transforms,
)

print(f"[INFO] found {len(trainDS)} examples in the training set...")
print(f"[INFO] found {len(validDS)} examples in the valid set...")

trainLoader = DataLoader(trainDS, shuffle=True,
    batch_size=config.BATCH_SIZE, pin_memory=config.PIN_MEMORY,
    num_workers=0)
validLoader = DataLoader(validDS, shuffle=False,
    batch_size=config.BATCH_SIZE, pin_memory=config.PIN_MEMORY,
    num_workers=0)

[INFO] found 3450 examples in the training set...
[INFO] found 100 examples in the valid set...


In [6]:
vdsr = VDSR().to(config.DEVICE)


vdsr.load_state_dict(torch.load('./output/model_epoch_13.pth'))
    
lossFunc = nn.MSELoss().to(config.DEVICE)


H = {"train_loss": [], "PSNR": []}

In [7]:
for epochs, lr in config.NUM_EPOCHS:
    opt = optim.Adam(vdsr.parameters(), lr=lr, weight_decay=config.WEIGHT_DECAY)
    train_time = 0.0
    validate_time = 0.0
    for epoch in range(epochs):
        start_time = time.time()
        vdsr.train()
        epoch_loss = 0.0
        epoch_acc = 0.0
        print("Epoch = {}, lr = {}".format(epoch, opt.param_groups[0]["lr"]))

        for iteration, (input, target) in enumerate(trainLoader, 1):
            input, target = input.to(config.DEVICE), target.to(config.DEVICE)
            opt.zero_grad()
            output = vdsr(input)
            loss = lossFunc(output, target)
            epoch_loss += loss.item()
            loss.backward()
            nn.utils.clip_grad_norm_(vdsr.parameters(), config.CLIP/lr)
            opt.step()

            print("===> Epoch[{}]({}/{}): Loss: {:.4f}".format(
                epoch, iteration, len(trainLoader), loss.item())
                 )
        print("===> Epoch {} Complete: Avg. Loss: {:.4f}".format(
            epoch, epoch_loss / len(trainLoader))
             )

        elapsed_time = time.time() - start_time
        train_time += elapsed_time

        print("===> {:.2f} seconds to train this epoch".format(elapsed_time))

        start_time = time.time()

        with torch.no_grad():
            vdsr.eval()
            avg_psnr = 0
            for (input, target) in validLoader:
                input, target = input.to(config.DEVICE), target.to(config.DEVICE)
                prediction = vdsr(input)
                mse = lossFunc(prediction, target)
                psnr = 10 * log10(1.0 / mse.item())
                avg_psnr += psnr
                print("===> Avg. PSNR: {:.4f} dB".format(
                    avg_psnr / len(validLoader))
                     )
        
            model_out_path = "output/model_epoch_{}.pth".format(epoch+14)
            torch.save(vdsr.state_dict(), model_out_path)

            print("===> average training time per epoch: {:.2f} seconds".format(train_time/epochs))
            print("===> average validation time per epoch: {:.2f} seconds".format(validate_time/epochs))
            print("===> training time: {:.2f} seconds".format(train_time))
            print("===> validation time: {:.2f} seconds".format(validate_time))
            print("===> total training time: {:.2f} seconds".format(train_time+validate_time))  

Epoch = 0, lr = 0.01
===> Epoch[0](1/432): Loss: 0.0314
===> Epoch[0](2/432): Loss: 0.0334
===> Epoch[0](3/432): Loss: 0.0314
===> Epoch[0](4/432): Loss: 0.0309
===> Epoch[0](5/432): Loss: 0.0438
===> Epoch[0](6/432): Loss: 0.0295
===> Epoch[0](7/432): Loss: 0.0386
===> Epoch[0](8/432): Loss: 0.0314
===> Epoch[0](9/432): Loss: 0.0258
===> Epoch[0](10/432): Loss: 0.0288
===> Epoch[0](11/432): Loss: 0.0355
===> Epoch[0](12/432): Loss: 0.0323
===> Epoch[0](13/432): Loss: 0.0298
===> Epoch[0](14/432): Loss: 0.0350
===> Epoch[0](15/432): Loss: 0.0445
===> Epoch[0](16/432): Loss: 0.0354
===> Epoch[0](17/432): Loss: 0.0354
===> Epoch[0](18/432): Loss: 0.0359
===> Epoch[0](19/432): Loss: 0.0415
===> Epoch[0](20/432): Loss: 0.0322
===> Epoch[0](21/432): Loss: 0.0418
===> Epoch[0](22/432): Loss: 0.0350
===> Epoch[0](23/432): Loss: 0.0278
===> Epoch[0](24/432): Loss: 0.0338
===> Epoch[0](25/432): Loss: 0.0311
===> Epoch[0](26/432): Loss: 0.0338
===> Epoch[0](27/432): Loss: 0.0356
===> Epoch[0](28

===> Epoch[0](225/432): Loss: 0.0371
===> Epoch[0](226/432): Loss: 0.0388
===> Epoch[0](227/432): Loss: 0.0329
===> Epoch[0](228/432): Loss: 0.0331
===> Epoch[0](229/432): Loss: 0.0325
===> Epoch[0](230/432): Loss: 0.0329
===> Epoch[0](231/432): Loss: 0.0325
===> Epoch[0](232/432): Loss: 0.0441
===> Epoch[0](233/432): Loss: 0.0383
===> Epoch[0](234/432): Loss: 0.0418
===> Epoch[0](235/432): Loss: 0.0422
===> Epoch[0](236/432): Loss: 0.0380
===> Epoch[0](237/432): Loss: 0.0310
===> Epoch[0](238/432): Loss: 0.0288
===> Epoch[0](239/432): Loss: 0.0368
===> Epoch[0](240/432): Loss: 0.0281
===> Epoch[0](241/432): Loss: 0.0299
===> Epoch[0](242/432): Loss: 0.0328
===> Epoch[0](243/432): Loss: 0.0304
===> Epoch[0](244/432): Loss: 0.0317
===> Epoch[0](245/432): Loss: 0.0400
===> Epoch[0](246/432): Loss: 0.0312
===> Epoch[0](247/432): Loss: 0.0272
===> Epoch[0](248/432): Loss: 0.0314
===> Epoch[0](249/432): Loss: 0.0373
===> Epoch[0](250/432): Loss: 0.0339
===> Epoch[0](251/432): Loss: 0.0284
=

===> Epoch[1](1/432): Loss: 0.0251
===> Epoch[1](2/432): Loss: 0.0360
===> Epoch[1](3/432): Loss: 0.0317
===> Epoch[1](4/432): Loss: 0.0328
===> Epoch[1](5/432): Loss: 0.0382
===> Epoch[1](6/432): Loss: 0.0386
===> Epoch[1](7/432): Loss: 0.0280
===> Epoch[1](8/432): Loss: 0.0277
===> Epoch[1](9/432): Loss: 0.0333
===> Epoch[1](10/432): Loss: 0.0388
===> Epoch[1](11/432): Loss: 0.0496
===> Epoch[1](12/432): Loss: 0.0258
===> Epoch[1](13/432): Loss: 0.0315
===> Epoch[1](14/432): Loss: 0.0357
===> Epoch[1](15/432): Loss: 0.0332
===> Epoch[1](16/432): Loss: 0.0292
===> Epoch[1](17/432): Loss: 0.0336
===> Epoch[1](18/432): Loss: 0.0321
===> Epoch[1](19/432): Loss: 0.0395
===> Epoch[1](20/432): Loss: 0.0321
===> Epoch[1](21/432): Loss: 0.0369
===> Epoch[1](22/432): Loss: 0.0282
===> Epoch[1](23/432): Loss: 0.0400
===> Epoch[1](24/432): Loss: 0.0287
===> Epoch[1](25/432): Loss: 0.0306
===> Epoch[1](26/432): Loss: 0.0288
===> Epoch[1](27/432): Loss: 0.0384
===> Epoch[1](28/432): Loss: 0.0339
=

===> Epoch[1](226/432): Loss: 0.0251
===> Epoch[1](227/432): Loss: 0.0355
===> Epoch[1](228/432): Loss: 0.0312
===> Epoch[1](229/432): Loss: 0.0376
===> Epoch[1](230/432): Loss: 0.0392
===> Epoch[1](231/432): Loss: 0.0435
===> Epoch[1](232/432): Loss: 0.0301
===> Epoch[1](233/432): Loss: 0.0454
===> Epoch[1](234/432): Loss: 0.0385
===> Epoch[1](235/432): Loss: 0.0296
===> Epoch[1](236/432): Loss: 0.0395
===> Epoch[1](237/432): Loss: 0.0346
===> Epoch[1](238/432): Loss: 0.0238
===> Epoch[1](239/432): Loss: 0.0357
===> Epoch[1](240/432): Loss: 0.0242
===> Epoch[1](241/432): Loss: 0.0294
===> Epoch[1](242/432): Loss: 0.0329
===> Epoch[1](243/432): Loss: 0.0271
===> Epoch[1](244/432): Loss: 0.0325
===> Epoch[1](245/432): Loss: 0.0311
===> Epoch[1](246/432): Loss: 0.0341
===> Epoch[1](247/432): Loss: 0.0332
===> Epoch[1](248/432): Loss: 0.0308
===> Epoch[1](249/432): Loss: 0.0324
===> Epoch[1](250/432): Loss: 0.0311
===> Epoch[1](251/432): Loss: 0.0364
===> Epoch[1](252/432): Loss: 0.0385
=

===> Epoch[2](1/432): Loss: 0.0365
===> Epoch[2](2/432): Loss: 0.0282
===> Epoch[2](3/432): Loss: 0.0314
===> Epoch[2](4/432): Loss: 0.0232
===> Epoch[2](5/432): Loss: 0.0394
===> Epoch[2](6/432): Loss: 0.0357
===> Epoch[2](7/432): Loss: 0.0309
===> Epoch[2](8/432): Loss: 0.0338
===> Epoch[2](9/432): Loss: 0.0240
===> Epoch[2](10/432): Loss: 0.0386
===> Epoch[2](11/432): Loss: 0.0428
===> Epoch[2](12/432): Loss: 0.0253
===> Epoch[2](13/432): Loss: 0.0308
===> Epoch[2](14/432): Loss: 0.0251
===> Epoch[2](15/432): Loss: 0.0335
===> Epoch[2](16/432): Loss: 0.0305
===> Epoch[2](17/432): Loss: 0.0337
===> Epoch[2](18/432): Loss: 0.0395
===> Epoch[2](19/432): Loss: 0.0250
===> Epoch[2](20/432): Loss: 0.0381
===> Epoch[2](21/432): Loss: 0.0375
===> Epoch[2](22/432): Loss: 0.0304
===> Epoch[2](23/432): Loss: 0.0358
===> Epoch[2](24/432): Loss: 0.0349
===> Epoch[2](25/432): Loss: 0.0351
===> Epoch[2](26/432): Loss: 0.0310
===> Epoch[2](27/432): Loss: 0.0373
===> Epoch[2](28/432): Loss: 0.0389
=

...

===> Epoch[2](411/432): Loss: 0.0411
===> Epoch[2](412/432): Loss: 0.0270
===> Epoch[2](413/432): Loss: 0.0375
===> Epoch[2](414/432): Loss: 0.0371
===> Epoch[2](415/432): Loss: 0.0337
===> Epoch[2](416/432): Loss: 0.0323
===> Epoch[2](417/432): Loss: 0.0311
===> Epoch[2](418/432): Loss: 0.0367
===> Epoch[2](419/432): Loss: 0.0429
===> Epoch[2](420/432): Loss: 0.0394
===> Epoch[2](421/432): Loss: 0.0351
===> Epoch[2](422/432): Loss: 0.0336
===> Epoch[2](423/432): Loss: 0.0366
===> Epoch[2](424/432): Loss: 0.0228
===> Epoch[2](425/432): Loss: 0.0365
===> Epoch[2](426/432): Loss: 0.0328
===> Epoch[2](427/432): Loss: 0.0297
===> Epoch[2](428/432): Loss: 0.0382
===> Epoch[2](429/432): Loss: 0.0327
===> Epoch[2](430/432): Loss: 0.0386
===> Epoch[2](431/432): Loss: 0.0289
===> Epoch[2](432/432): Loss: 0.0330
===> Epoch 2 Complete: Avg. Loss: 0.0332
===> 1640.33 seconds to train this epoch
===> Avg. PSNR: 1.1191 dB
===> Avg. PSNR: 2.3288 dB
===> Avg. PSNR: 3.4741 dB
===> Avg. PSNR: 4.6160 dB


===> Epoch[3](186/432): Loss: 0.0398
===> Epoch[3](187/432): Loss: 0.0405
===> Epoch[3](188/432): Loss: 0.0374
===> Epoch[3](189/432): Loss: 0.0350
===> Epoch[3](190/432): Loss: 0.0362
===> Epoch[3](191/432): Loss: 0.0330
===> Epoch[3](192/432): Loss: 0.0295
===> Epoch[3](193/432): Loss: 0.0354
===> Epoch[3](194/432): Loss: 0.0411
===> Epoch[3](195/432): Loss: 0.0364
===> Epoch[3](196/432): Loss: 0.0325
===> Epoch[3](197/432): Loss: 0.0281
===> Epoch[3](198/432): Loss: 0.0312
===> Epoch[3](199/432): Loss: 0.0308
===> Epoch[3](200/432): Loss: 0.0318
===> Epoch[3](201/432): Loss: 0.0287
===> Epoch[3](202/432): Loss: 0.0280
===> Epoch[3](203/432): Loss: 0.0308
===> Epoch[3](204/432): Loss: 0.0344
===> Epoch[3](205/432): Loss: 0.0268
===> Epoch[3](206/432): Loss: 0.0331
===> Epoch[3](207/432): Loss: 0.0271
===> Epoch[3](208/432): Loss: 0.0383
===> Epoch[3](209/432): Loss: 0.0329
===> Epoch[3](210/432): Loss: 0.0408
===> Epoch[3](211/432): Loss: 0.0339
===> Epoch[3](212/432): Loss: 0.0384
=

===> Epoch[3](408/432): Loss: 0.0278
===> Epoch[3](409/432): Loss: 0.0262
===> Epoch[3](410/432): Loss: 0.0352
===> Epoch[3](411/432): Loss: 0.0361
===> Epoch[3](412/432): Loss: 0.0334
===> Epoch[3](413/432): Loss: 0.0293
===> Epoch[3](414/432): Loss: 0.0487
===> Epoch[3](415/432): Loss: 0.0358
===> Epoch[3](416/432): Loss: 0.0322
===> Epoch[3](417/432): Loss: 0.0351
===> Epoch[3](418/432): Loss: 0.0312
===> Epoch[3](419/432): Loss: 0.0439
===> Epoch[3](420/432): Loss: 0.0361
===> Epoch[3](421/432): Loss: 0.0402
===> Epoch[3](422/432): Loss: 0.0355
===> Epoch[3](423/432): Loss: 0.0389
===> Epoch[3](424/432): Loss: 0.0285
===> Epoch[3](425/432): Loss: 0.0398
===> Epoch[3](426/432): Loss: 0.0325
===> Epoch[3](427/432): Loss: 0.0358
===> Epoch[3](428/432): Loss: 0.0360
===> Epoch[3](429/432): Loss: 0.0318
===> Epoch[3](430/432): Loss: 0.0304
===> Epoch[3](431/432): Loss: 0.0201
===> Epoch[3](432/432): Loss: 0.0399
===> Epoch 3 Complete: Avg. Loss: 0.0333
===> 1640.63 seconds to train this

===> Epoch[4](183/432): Loss: 0.0311
===> Epoch[4](184/432): Loss: 0.0333
===> Epoch[4](185/432): Loss: 0.0348
===> Epoch[4](186/432): Loss: 0.0391
===> Epoch[4](187/432): Loss: 0.0369
===> Epoch[4](188/432): Loss: 0.0249
===> Epoch[4](189/432): Loss: 0.0330
===> Epoch[4](190/432): Loss: 0.0299
===> Epoch[4](191/432): Loss: 0.0266
===> Epoch[4](192/432): Loss: 0.0195
===> Epoch[4](193/432): Loss: 0.0358
===> Epoch[4](194/432): Loss: 0.0334
===> Epoch[4](195/432): Loss: 0.0340
===> Epoch[4](196/432): Loss: 0.0328
===> Epoch[4](197/432): Loss: 0.0314
===> Epoch[4](198/432): Loss: 0.0333
===> Epoch[4](199/432): Loss: 0.0352
===> Epoch[4](200/432): Loss: 0.0322
===> Epoch[4](201/432): Loss: 0.0361
===> Epoch[4](202/432): Loss: 0.0306
===> Epoch[4](203/432): Loss: 0.0316
===> Epoch[4](204/432): Loss: 0.0336
===> Epoch[4](205/432): Loss: 0.0255
===> Epoch[4](206/432): Loss: 0.0306
===> Epoch[4](207/432): Loss: 0.0278
===> Epoch[4](208/432): Loss: 0.0270
===> Epoch[4](209/432): Loss: 0.0412
=

===> Epoch[4](405/432): Loss: 0.0291
===> Epoch[4](406/432): Loss: 0.0298
===> Epoch[4](407/432): Loss: 0.0263
===> Epoch[4](408/432): Loss: 0.0423
===> Epoch[4](409/432): Loss: 0.0279
===> Epoch[4](410/432): Loss: 0.0375
===> Epoch[4](411/432): Loss: 0.0344
===> Epoch[4](412/432): Loss: 0.0382
===> Epoch[4](413/432): Loss: 0.0337
===> Epoch[4](414/432): Loss: 0.0355
===> Epoch[4](415/432): Loss: 0.0346
===> Epoch[4](416/432): Loss: 0.0282
===> Epoch[4](417/432): Loss: 0.0370
===> Epoch[4](418/432): Loss: 0.0387
===> Epoch[4](419/432): Loss: 0.0383
===> Epoch[4](420/432): Loss: 0.0321
===> Epoch[4](421/432): Loss: 0.0425
===> Epoch[4](422/432): Loss: 0.0360
===> Epoch[4](423/432): Loss: 0.0366
===> Epoch[4](424/432): Loss: 0.0273
===> Epoch[4](425/432): Loss: 0.0363
===> Epoch[4](426/432): Loss: 0.0317
===> Epoch[4](427/432): Loss: 0.0312
===> Epoch[4](428/432): Loss: 0.0344
===> Epoch[4](429/432): Loss: 0.0408
===> Epoch[4](430/432): Loss: 0.0388
===> Epoch[4](431/432): Loss: 0.0352
=